# This is a very simple implementation of a model that counts the number of particles that are created in a scattering event. 
## The data comes from the LHC where beams of protons and antiprotons are being collided. After each collision event a detector built of several layers is used to track the created particles. The model takes all hits of particles in the detector - in a (x,y,z)-coordinate system - and outputs the number of created particles: a integer number between 0 and 15000, which was my threshold for the maximal number of particles per event. It is a classification problem with a huge number of classes. It might be sensible to make some step-size of 20, or maybe even 100, to reduce the number of classes. This can be decided at a later point. (This project was a part of my work for the Kaggle competition "TrackML Particle Tracking Challenge" )

In [ ]:
import pandas as pd
import numpy as np
from trackml.dataset import load_event
from keras import models, layers
from keras.utils import np_utils

## Load one event to get some insight in the data

In [ ]:
hits, cells, particles, truth = load_event('train_100_events/event000001000')

## Transform the detector data into a Euclidean coordinate system, a discretized box of size 100 x 100 x 100

In [ ]:
#we need to transform our data in to a det_shape = (x,y,z) form... 
#det_shape = (1000,1000,1000)
#x_max = ymax = 1025 
#z_max = 3000
def trans_input(hits_input = hits, det_shape = (100,100,100), x_max = 1025,ymax = 1025, z_max = 3000):
    hits_input[["x","y"]] = (hits[["x","y"]] + x_max )/(2*x_max) * (det_shape[0]-1)
    hits_input[["z"]] = (hits[["z"]] + z_max )/(2*z_max) * (det_shape[2]-1)
    hits_input[["x","y","z"]] = hits_input[["x","y","z"]].astype(int)
    detector_hits = np.zeros(det_shape)
    for each_hit in range( len(hits) ):
        xx,yy,zz = hits_input.loc[each_hit][["x","y","z"]]
        detector_hits[ xx,yy,zz ] = detector_hits[ xx,yy,zz ] + 1;
    return detector_hits

In [ ]:
new_hits = trans_input()

In [ ]:
print(new_hits.mean())

## Load the first 100 events. The data is quite big, so let us start with 100 to make some tests and load the rest later.

In [ ]:
def load_ev(i):
    if i < 10:
        hits, _, _, truth = load_event('train_100_events/event00000100'+str(i))
    else:
        hits, _, _, truth = load_event('train_100_events/event0000010'+str(i))
    return hits, truth

max_particles = 15000
x_train = new_hits
y_train = np.zeros((100,max_particles))
y_train[0,len( truth.particle_id.unique() )] = 1

for i in range(1,100):
    hits, truth = load_ev(i)
    if i == 1: 
        x_train = np.vstack(([x_train], [trans_input(hits)]) )
        y_train[i,len( truth.particle_id.unique() )] = 1
    else:
        x_train = np.vstack((x_train, [trans_input(hits)] )) 
        y_train[i,len( truth.particle_id.unique() )] = 1


In [ ]:
print(y_train.shape,x_train.shape)

## Now let me define a model for classification: It is inspired from image classification, so I have some convolution layers (3D in this case) and one fully connected layer in the end.

In [3]:
# Let us try some model, set the number of maximal particles to 15000 (empirical value)
def counting_model(det_shape, max_particles=15000, optimizer='Nadam'):
    """
    A very simple convolution model that predicts how many particles are in each event.
    Output is discrete num_particles class, set up as classification.
    """
    inputs = layers.Input(shape=det_shape)
    # Reshape to add a channel dimension to the input
    hidden = layers.Reshape((1,) + det_shape)(inputs)
    # Convolutions and downsampling
    hidden = layers.Conv3D(8, (3, 3,3), padding='same', activation='relu')(hidden)
    hidden = layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same')(hidden)
    hidden = layers.Conv3D(12, (3, 3, 3), padding='same', activation='relu')(hidden)
    hidden = layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same')(hidden)
    hidden = layers.Conv3D(16, (3, 3, 3), padding='same', activation='relu')(hidden)
    hidden = layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same')(hidden)
    hidden = layers.Conv3D(20, (3, 3, 3), padding='same', activation='relu')(hidden)
    hidden = layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same')(hidden)
    hidden = layers.Conv3D(24, (3, 3, 3), padding='same', activation='relu')(hidden)
    # Fully connected and softmax
    hidden = layers.Flatten()(hidden)
    outputs = layers.Dense(max_particles, activation='softmax')(hidden)
    # Compile the model
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer, metrics=['accuracy'])
    return model


In [ ]:
model = counting_model(det_shape= (100,100,100))

## Train the model (Here only with the first 100 events)

In [ ]:
history = model.fit(x_train, y_train, steps_per_epoch=5, epochs=100)

## For this small set of events the model start to overfit very fast. Let me see how valuable the model is on new data before I continue to train it on the large dataset.

In [ ]:
def load_test(i):
    if i < 10:
        hits, _, _, truth = load_event('train_200_events/event00000110'+str(i))
    else:
        hits, _, _, truth = load_event('train_200_events/event0000011'+str(i))
    return hits, truth

max_particles = 15000
hits, _, _, truth = load_event('train_200_events/event000001100')
x_test = trans_input(hits)
y_test = np.zeros((100,max_particles))
y_test[0,len( truth.particle_id.unique() )] = 1

for i in range(1,100):
    hits, truth = load_test(i)
    if i == 1: 
        x_test = np.vstack(([x_test], [trans_input(hits)]) )
        y_test[i,len( truth.particle_id.unique() )] = 1
    else:
        x_test = np.vstack((x_test, [trans_input(hits)] )) 
        y_test[i,len( truth.particle_id.unique() )] = 1


In [ ]:
eval_model = model.evaluate(x_test, y_test)

In [ ]:
eval_model